#### Fine tuning (Transfer learning)

In [27]:
import os
from keras.preprocessing import image
import numpy as np

In [2]:
path = "/Users/apoorvgarg/Downloads/Coding/machine-learning-online-2018/Datasets/Images/Images"
folders = os.listdir(path)

In [3]:
image_data = []
labels = []
label_dict = {
    "cats":0,
    "dogs":1,
    "horses":2,
    "humans":3
}

In [4]:
for f in folders:
    if f == '.DS_Store': continue
    f_path = os.path.join(path,f)
    for im in os.listdir(f_path):
        img = image.load_img(os.path.join(f_path,im),target_size=((224,224)))
        img_array = image.img_to_array(img)
        image_data.append(img_array)
        labels.append(label_dict[f])

In [5]:
len(image_data)

724

In [6]:
image_data = np.array(image_data)

In [7]:
image_data.shape

(724, 224, 224, 3)

In [8]:
combined = list(zip(image_data,labels))
np.random.shuffle(combined)

In [9]:
image_data,labels = zip(*combined)

In [10]:
print(labels[:5])

(1, 3, 3, 2, 2)


In [11]:
X_train = np.array(image_data)
Y_train = np.array(labels)

print(X_train.shape,Y_train.shape)


(724, 224, 224, 3) (724,)


In [12]:
def one_hot(Y):
    noc = len(np.unique(Y))
    y_ = np.zeros((Y.shape[0],noc),dtype=np.int8)
    for ix,x in enumerate(Y):
        y_[ix,x] = 1
    return y_

In [13]:
Y = one_hot(Y_train)

In [14]:
from keras.utils import np_utils
Y_train = np_utils.to_categorical(Y_train)

In [15]:
print(Y_train.shape)

(724, 4)


In [16]:
from keras.applications import ResNet50
from keras.optimizers import Adam
from keras.layers import *
from keras.models import Model

from matplotlib import pyplot as plt



In [17]:
model = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3))

In [18]:
# model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [19]:
av1 = GlobalAveragePooling2D()(model.output)
fc1 = Dense(256,activation='relu')(av1)
d1 = Dropout(0.5)(fc1)
fc2 = Dense(4,activation='softmax')(d1)


model_new = Model(inputs=model.input,outputs=fc2)

In [20]:
# model_new.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

## feature extraction
- Convent_base non-trainable


## Fine tuning
- Learning rate must be small


In [21]:
adam = Adam(lr=0.00003)
model_new.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])

In [22]:
# for ix in range(len(model_new.layers)):
#     print(ix,model_new.layers[ix])

In [23]:
## from layer 168 we will fine tune
## layers till 168 will be set non trainable


In [24]:
for ix in range(169):
    model_new.layers[ix].trainable = False

model_new.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])

In [25]:
# model_new.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [26]:
hist = model_new.fit(X_train,Y_train,shuffle=True,batch_size=16,epochs=5,validation_split=0.20)

Epoch 1/5
37/37 [==============================] - 60s 2s/step - loss: 1.3101 - accuracy: 0.4646 - val_loss: 0.4949 - val_accuracy: 0.8759